<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/present/youtube/render/optical-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import math
import cv2

from PIL import Image,ImageDraw
from matplotlib import pyplot as plt

RESOLUTION_4K = (3840, 2160)
RESOLUTION_HD = (1920, 1080)
RESOLUTION_SD = (640, 480)
RESOLUTION_IG = (2048, 2048)

def inside_rect(pt,r):
  return pt[0]>=r[0][0] and pt[0]<=r[1][0] and pt[1]>=r[0][1] and pt[1]<=r[1][1]

def render_circle(diameter,from_color="#ff0000",to_color="#ffff00"):
  im = Image.new(mode='RGB', size=(diameter,diameter), color=from_color)
  
  # Radial alpha/transparency layer
  y = np.linspace(-0.9, 0.25, diameter)[None, :]*255
  x = np.linspace(-0.25, 0.9, diameter)[:, None]*255
  alpha = np.sqrt(y**2 + x**2)
  alpha =  np.clip(0,255,alpha)

  # Apply alpha to the to color
  im.putalpha(Image.fromarray(alpha.astype(np.uint8)))
  im2 = Image.new(mode='RGB', size=(diameter,diameter), color=to_color)
  im2.paste(im,(0,0),im)

  # Mask the final circle
  circle_mask = Image.new(mode='L', size=(diameter,diameter), color=0)
  draw = ImageDraw.Draw(circle_mask)
  draw.ellipse((2,2,diameter-4,diameter-4), fill=255)
  im2.putalpha(circle_mask)
  return im2


class OpticalIllusion:
  def __init__(self, r, circle_count, bars, circle_size, bar_colors, from_color,to_color):
    self.width = r[0]
    self.height = r[1]
    self.circle_count = circle_count
    self.bar_colors = bar_colors
    self.diameter = int(self.width * circle_size)
    self.img = Image.new(mode='RGB', size=(self.width,self.height), color=(255,0,0))
    self.draw = ImageDraw.Draw(self.img)
    self.bars = np.linspace(0, self.height, num=bars+1, endpoint=True, retstep=False, dtype=int, axis=0)
    self.circles = []
    self.colors = (list(range(len(self.bar_colors))) * bars)[:bars]
    self.circle_img = render_circle(self.diameter,from_color=from_color,to_color=to_color)

  def place_circles(self):
    self.circles = []

    for c in range(0,len(self.bar_colors)):
      for i in range(0,self.circle_count):
        d = False
        while not d:
          x = np.random.randint(0,self.width-self.diameter)
          y = np.random.randint(0,self.height-self.diameter)
          circle_rect = [(x,y),(x+self.diameter,y+self.diameter)]
          if not self.too_close(circle_rect):
            self.circles.append({'rect':circle_rect,'color':c})
            d = True

  def draw_circle(self,r,c,circle_img):
    self.img.paste(circle_img,r[0],circle_img)

    idx1 = np.argmax(self.bars>r[0][1])-1
    idx2 = np.argmax(self.bars>r[1][1])-1

    for idx in range(idx1,idx2+1):
      yy = self.bars[idx1]
      color_idx = self.colors[idx]
      if color_idx == c:
        x = r[0][0]
        self.draw.rectangle([(x,self.bars[idx]), (x+self.diameter,self.bars[idx+1]-1)], fill=self.bar_colors[color_idx])

  def render(self):
    for i in range(len(self.bars)-1):
      x1 = 0
      x2 = self.width
      y1 = self.bars[i]
      y2 = self.bars[i+1]-1
      self.draw.rectangle([(x1,y1), (x2,y2)], fill=self.bar_colors[self.colors[i]])

    all_circles = []

    for circle in self.circles:
      r = circle['rect']
      c = circle['color']
      self.draw_circle(r,c,self.circle_img)

  def too_close(self,r):
    for circle in self.circles:
      r2 = circle['rect']
      c = circle['color']
      p1 = [r[0][0],r[1][1]]
      p2 = [r[1][0],r[0][1]]
      if inside_rect(r[0],r2) | inside_rect(r[1],r2)| inside_rect(p1,r2)| inside_rect(p2,r2):
        return True
    return False

  def save(self,filename):
    self.img.save(filename)
        
MODE = 'JEFF'

if MODE=='NOVICK':
  optic = OpticalIllusion(
      RESOLUTION_4K,
      circle_count=3, bars=90,circle_size=0.12,
      from_color="#4e3928",to_color="#ffc48e",
      bar_colors=["#fa0600","#50d725","#0b81ff"])
elif MODE=='JEFF':
    optic = OpticalIllusion(
      RESOLUTION_SD,
      circle_count=3, bars=90,circle_size=0.12,
      from_color="#303020",to_color="#a0a020",
      bar_colors=["#a0a000","#a000a0","#00a0a0"])

optic.place_circles()
optic.render()
optic.save("optic.png")

In [ ]:
from google.colab import files
files.download('/content/movie.mp4') 